In [1]:
%%time
from main import *

raw_courses = read('data/courses.txt')
raw_industries = read('data/industries.txt')
raw_job_industries = read('data/job_industries.txt')

Wall time: 199 ms


In [124]:
%%time
# note: enchant module and library should be installed
from clean import clean

clean_courses = clean(raw_courses, 'course', 'data/course_dictionary.txt')
clean_industries = clean(raw_industries, 'industry', 'data/industry_dictionary.txt')
clean_job_industries = clean(raw_job_industries, 'industry', 'data/industry_dictionary.txt')

Wall time: 11.3 s


In [125]:
%%time
# note: stemming module should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

Wall time: 62 ms


In [126]:
sorted(course_clusters)[:10]

['ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURE',
 'ANIMATION',
 'APPLIED',
 'ARCHITECTURE',
 'ART',
 'ASSOCIATES']

In [127]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BPO',
 'BROKERAGE',
 'BUILDING']

In [128]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALLCENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION',
 'ENGINE']

In [229]:
%%time

import analysis
reload(analysis)
from analysis import *

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = likelihood_matrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

Wall time: 35 ms


In [230]:
CourseIndustry.dataframe.head()

,REPAIR,SUPPLY,ENGINE,FINANCE,SOLUTIONS,RESTAURANT,ELECTRONIC,PARTS,DEVELOPMENT,LINE,...,FORWARDING,BANKING,SELLING,WRITING,PLANT,FABRICATION,CITY,HEALTH,DISTRIBUTOR,EQUIPMENT
ANIMATION,0,0,0.000000,0,0.107545,0,0.000000,0,0.000000,0.000000,...,0,0,0,0,0,0.174595,0,0,0,0.000000
ENGINEER,0,0,0.764188,0,0.000000,0,0.000000,0,0.000000,0.118523,...,0,0,0,0,0,0.000000,0,0,0,0.000000
ELEMENTARY,0,0,0.000000,0,0.000000,0,0.060941,0,0.109915,0.000000,...,0,0,0,0,0,0.000000,0,0,0,0.126877
FINANCE,0,0,0.000000,1,0.000000,0,0.000000,0,0.000000,0.000000,...,0,0,0,0,0,0.000000,0,0,0,0.000000
LITERATURE,0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0.000000,...,0,0,0,0,0,0.000000,0,0,0,0.000000


In [231]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

Wall time: 21 ms


In [232]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

Wall time: 50 ms


In [233]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

Wall time: 0 ns


BUSINESS        1.000000
ACCOUNTING      1.000000
AGRIBUSINESS    0.747282
WRITING         0.159128
DRAFTING        0.135685
MARKETING       0.124098
NURSING         0.117439
ADVERTISING     0.082212
MARINE          0.074211
MEDICINE        0.060449
Name: BUSINESS, dtype: float64

In [234]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

Wall time: 0 ns


RESEARCH        0.945460
INFORMATION     0.627905
TECHNOLOGY      0.464770
CONSULTING      0.426784
EDUCATION       0.243047
CONSTRUCTION    0.239345
Name: SCIENCE, dtype: float64

In [235]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

Wall time: 0 ns


In [236]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

Wall time: 0 ns


In [237]:
# rank recommendations with the highest percentage
matches.nlargest(5)

RESEARCH       67.752414
FINANCE         8.919492
FINANCIAL       5.763232
RESTAURANT      4.416795
MAINTENANCE     3.333823
dtype: float64

In [238]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

Wall time: 0 ns


56.995991163098033

In [239]:
# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = likelihood_matrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

In [240]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

0.031577389938425966